In [5]:
import os
import h5py
import numpy as np

In [6]:
if 'NERSC_HOST' not in os.environ:
    dir_cmx = '/Users/ChangHoon/data/feasiBGS/cmx/'
    dir_sky = '/Users/ChangHoon/data/feasiBGS/sky/'
else: 
    dir_cmx = '/global/cfs/cdirs/desi/users/chahah/bgs_exp_coadd/'
    dir_sky = '/global/cfs/cdirs/desi/users/chahah/sky/'

In [11]:
fsky = h5py.File(os.path.join(dir_cmx, 'sky_fibers.cmx.v1.hdf5'), 'r')
    
sky_data = {}
for k in fsky.keys():
    sky_data[k] = fsky[k][...]

In [13]:
def read_cmx_skies(): 
    ''' read CMX sky fibers and return median sky surface brightness measurements for each exposure
    '''
    fsky = h5py.File(os.path.join(dir_cmx, 'sky_fibers.cmx.v1.hdf5'), 'r')
    
    sky_data = {}
    for k in fsky.keys():
        sky_data[k] = fsky[k][...]
        
    bad_seeing = (sky_data['tileid'] == 70502) | (sky_data['date'] == 20200314) #bad seeing on Feb 25 and 27

    exp_cuts = ~bad_seeing

    for k in sky_data.keys(): 
        if 'wave' not in k: 
            sky_data[k] = sky_data[k][exp_cuts]
        else:
            sky_data[k] = sky_data[k]
            
    uniq_exps, i_uniq = np.unique(sky_data['expid'], return_index=True)

    # compile median observing conditions for each unique exposure and
    # get the median sky fluxes of all sky fibers
    sky_uniq_exps = {} 
    for k in ['date', 'tileid', 'expid', 
              'airmass', 'moon_ill', 'moon_alt', 'moon_sep', 
              'sun_alt', 'sun_sep', 'exptime', 
              'sky_ratio_5000']: 
        sky_uniq_exps[k] = np.zeros(len(uniq_exps))

    sky_uniq_exps['wave_b'] = sky_data['wave_b']
    sky_uniq_exps['wave_r'] = sky_data['wave_r']
    sky_uniq_exps['wave_z'] = sky_data['wave_z']
    
    sky_uniq_exps['sky_b'] = np.zeros((len(uniq_exps), len(sky_data['wave_b'])))
    sky_uniq_exps['sky_r'] = np.zeros((len(uniq_exps), len(sky_data['wave_r'])))
    sky_uniq_exps['sky_z'] = np.zeros((len(uniq_exps), len(sky_data['wave_z'])))
    
    print('date \t\t tile \t exp \t texp \t airmass \t moon_ill \t moon_alt \t moon_sep')
    for _i, _i_uniq, _exp in zip(range(len(i_uniq)), i_uniq, uniq_exps): 
        _is_exp = (sky_data['expid'] == _exp)

        sky_uniq_exps['date'][_i] = sky_data['date'][_i_uniq]
        sky_uniq_exps['tileid'][_i] = sky_data['tileid'][_i_uniq]
        sky_uniq_exps['expid'][_i] = sky_data['expid'][_i_uniq]
        sky_uniq_exps['airmass'][_i] = np.median(sky_data['airmass'][_is_exp])
        sky_uniq_exps['moon_ill'][_i] = np.median(sky_data['moon_ill'][_is_exp])
        sky_uniq_exps['moon_alt'][_i] = np.median(sky_data['moon_alt'][_is_exp])
        sky_uniq_exps['moon_sep'][_i] = np.median(sky_data['moon_sep'][_is_exp])
        sky_uniq_exps['exptime'][_i] = sky_data['exptime'][_is_exp][0]

        sky_uniq_exps['sun_alt'][_i] = np.median(sky_data['sun_alt'][_is_exp])
        sky_uniq_exps['sun_sep'][_i] = np.median(sky_data['sun_sep'][_is_exp])        
        
        sky_uniq_exps['sky_b'][_i] = np.median(sky_data['sky_sb_b'][_is_exp], axis=0) 
        sky_uniq_exps['sky_r'][_i] = np.median(sky_data['sky_sb_r'][_is_exp], axis=0) 
        sky_uniq_exps['sky_z'][_i] = np.median(sky_data['sky_sb_z'][_is_exp], axis=0)
        
        sky_uniq_exps['sky_ratio_5000'][_i] = np.median(sky_data['sky_ratio_5000'][_is_exp])

        print('%i \t %i \t %i \t %.f \t %.2f \t\t %.2f \t\t %.1f \t\t %f \t\t %f' % 
              (sky_data['date'][_i_uniq], sky_data['tileid'][_i_uniq], sky_data['expid'][_i_uniq], 
               sky_uniq_exps['exptime'][_i], 
               sky_uniq_exps['airmass'][_i], sky_uniq_exps['moon_ill'][_i], 
               sky_uniq_exps['moon_alt'][_i], sky_uniq_exps['moon_sep'][_i], sky_uniq_exps['sky_ratio_5000'][_i]))

    return sky_uniq_exps

In [14]:
cmx_skies = read_cmx_skies()

date 		 tile 	 exp 	 texp 	 airmass 	 moon_ill 	 moon_alt 	 moon_sep
20200303 	 70500 	 52932 	 500 	 1.15 		 0.65 		 35.8 		 34.692321 		 3.488880
20200303 	 70500 	 52934 	 500 	 1.19 		 0.65 		 31.0 		 34.557860 		 3.541012
20200303 	 70500 	 52936 	 500 	 1.25 		 0.65 		 26.2 		 34.423687 		 3.612682
20200303 	 70500 	 52938 	 500 	 1.31 		 0.65 		 21.6 		 34.292890 		 3.679804
20200303 	 70500 	 52940 	 500 	 1.39 		 0.65 		 17.0 		 34.162246 		 3.741839
20200303 	 70500 	 52942 	 500 	 1.49 		 0.66 		 12.2 		 34.026271 		 3.670622
20200304 	 70510 	 53078 	 500 	 1.43 		 0.74 		 55.8 		 79.209427 		 3.288664
20200304 	 70510 	 53080 	 500 	 1.34 		 0.75 		 52.1 		 79.061903 		 3.014404
20200304 	 70510 	 53081 	 500 	 1.29 		 0.75 		 49.2 		 78.944160 		 2.865382
20200304 	 70510 	 53082 	 500 	 1.24 		 0.75 		 46.4 		 78.827312 		 2.743923
20200304 	 70510 	 53083 	 500 	 1.19 		 0.75 		 43.5 		 78.710167 		 2.639909
20200304 	 70510 	 53084 	 500 	 1.16 		 0.75 		 40.7 		 78.59

Seems like we should run VI on **tile 70500**